In [1]:
"""
Source: https://www.pinecone.io/learn/series/langchain/langchain-tools/
"""
None

In [2]:
from langchain.tools import BaseTool
from math import pi
from typing import Union
import random
from pprint import pprint
import json


class CircumferenceTool(BaseTool):
    name = "Circumference calculator"
    description = "use this tool when you need to calculate a circumference using the radius of a circle"

    def _run(self, radius: Union[int, float]):
        return float(radius) * 2.0 * pi

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")


class GetAdvertiserIdTool(BaseTool):
    name = "Get Advertiser Id"
    description = "Get Advertiser Id out of a company, advertiser, parent company name"

    def _run(self, client: str) -> int:
        return random.randint(1, 1000)


class GetClientSpentHistoryTool(BaseTool):
    name = "Client Spent History"
    description = (
        "use this tool to get the historical spent (also called revenue) from an advertiser, "
        "the input for this function is the advertiser_id, "
        "if you don't have it use lookup the advertiser id using the GetAdvertiserIdTool"
    )

    def _run(self, advertiser_id: int):
        return {
            "date": [
                "2020-jan",
                "2020-fev",
                "2020-mar",
                "2020-apr",
                "2020-may",
                "2020-jun",
            ],
            "spent": [
                random.randint(100, 120),
                random.randint(120, 150),
                150,
                100,
                90,
                100,
            ],
        }

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# initialize LLM (we use ChatOpenAI because we'll later define a `chat` agent)
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

In [4]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = [GetAdvertiserIdTool(), CircumferenceTool(), GetClientSpentHistoryTool()]

In [5]:
agent = initialize_agent(
    # agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10,
    early_stopping_method="generate",
    memory=conversational_memory,
)
agent("How much did Amazon, spent in February?")



> Entering new AgentExecutor chain...
{
    "action": "Client Spent History",
    "action_input": "Amazon"
}
Observation: {'date': ['2020-jan', '2020-fev', '2020-mar', '2020-apr', '2020-may', '2020-jun'], 'spent': [117, 134, 150, 100, 90, 100]}
Thought:{
    "action": "Final Answer",
    "action_input": "Amazon spent $134 in February."
}

> Finished chain.


{'input': 'How much did Amazon, spent in February?',
 'chat_history': [],
 'output': 'Amazon spent $134 in February.'}

In [6]:
agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    # agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10,
    # early_stopping_method="generate",
    memory=conversational_memory,
)
agent("How much did Amazon spent in February?")



> Entering new AgentExecutor chain...
I need to find the advertiser_id for Amazon and then use the Client Spent History tool to get the historical spent for February.
Action: Get Advertiser Id
Action Input: Company Name - Amazon
Observation: 659
Thought:I have the advertiser_id for Amazon. Now I can use the Client Spent History tool to get the historical spent for February.
Action: Client Spent History
Action Input: Advertiser Id - 659
Observation: {'date': ['2020-jan', '2020-fev', '2020-mar', '2020-apr', '2020-may', '2020-jun'], 'spent': [114, 141, 150, 100, 90, 100]}
Thought:The historical spent for Amazon in February is 141.
Final Answer: 141

> Finished chain.


{'input': 'How much did Amazon spent in February?',
 'chat_history': [HumanMessage(content='How much did Amazon, spent in February?', additional_kwargs={}, example=False),
  AIMessage(content='Amazon spent $134 in February.', additional_kwargs={}, example=False)],
 'output': '141'}

In [7]:
agent = initialize_agent(
    # agent=AgentType.OPENAI_MULTI_FUNCTIONS,
    # agent="chat-conversational-react-description",
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10,
    early_stopping_method="generate",
    memory=conversational_memory,
)
try:
    agent("How does Amazon and Google revenue compares in january?")
except Exception as e:
    print(e)
# This one seems too much for the models to correlate



> Entering new AgentExecutor chain...
I need to find the revenue for both Amazon and Google in January.
Action: Get Advertiser Id
Action Input: Amazon
Observation: 414
Thought:I have the advertiser id for Amazon.
Action: Client Spent History
Action Input: 414 (advertiser id for Amazon)
Observation: {'date': ['2020-jan', '2020-fev', '2020-mar', '2020-apr', '2020-may', '2020-jun'], 'spent': [103, 136, 150, 100, 90, 100]}
Thought:I have the spent history for Amazon in January.
Action: Get Advertiser Id
Action Input: Google
Observation: 955
Thought:I have the advertiser id for Google.
Action: Client Spent History
Action Input: 955 (advertiser id for Google)
Observation: {'date': ['2020-jan', '2020-fev', '2020-mar', '2020-apr', '2020-may', '2020-jun'], 'spent': [107, 131, 150, 100, 90, 100]}
Thought:Could not parse LLM output: `I have the spent history for Google in January as well.
Action: None`


In [8]:
pprint(json.loads(agent.json()), width=120, compact=True)

{'agent': {'_type': 'zero-shot-react-description',
           'allowed_tools': ['Get Advertiser Id', 'Circumference calculator', 'Client Spent History'],
           'llm_chain': {'_type': 'llm_chain',
                         'llm': {'_type': 'openai-chat',
                                 'max_tokens': None,
                                 'model': 'gpt-3.5-turbo',
                                 'model_name': 'gpt-3.5-turbo',
                                 'n': 1,
                                 'request_timeout': None,
                                 'stream': False,
                                 'temperature': 0.0},
                         'llm_kwargs': {},
                         'memory': None,
                         'metadata': None,
                         'output_key': 'text',
                         'output_parser': {'_type': 'default'},
                         'prompt': {'_type': 'prompt',
                                    'input_variables': ['input', 'agen

In [9]:
def print_agent(bob, level=0):
    index = "|_" * level

    def print_attr(bob, attr):
        if hasattr(bob, attr):
            print(f"{index} bob.{attr}: {getattr(bob, attr)}")

    print(f"{index}" + "-" * (80 - len(index)))

    print_attr(bob, "__class__")

    if hasattr(bob, "__dict__"):
        for key, value in bob.__dict__.items():
            if isinstance(value, (str, int, float, bool)) or value is None:
                if key == "openai_api_key":
                    value = "REDACTED"
                print(f"{index} Key: {key} = {value}")
            else:
                print_agent(value, level + 1)
    # print(f"{index}:: {bob}")


print_agent(agent)

--------------------------------------------------------------------------------
 bob.__class__: <class 'langchain.agents.agent.AgentExecutor'>
|_------------------------------------------------------------------------------
|_ bob.__class__: <class 'langchain.memory.buffer_window.ConversationBufferWindowMemory'>
|_|_----------------------------------------------------------------------------
|_|_ bob.__class__: <class 'langchain.memory.chat_message_histories.in_memory.ChatMessageHistory'>
|_|_|_--------------------------------------------------------------------------
|_|_|_ bob.__class__: <class 'list'>
|_ Key: output_key = None
|_ Key: input_key = None
|_ Key: return_messages = True
|_ Key: human_prefix = Human
|_ Key: ai_prefix = AI
|_ Key: memory_key = chat_history
|_ Key: k = 5
 Key: callbacks = None
 Key: callback_manager = None
 Key: verbose = True
|_------------------------------------------------------------------------------
|_ bob.__class__: <class 'list'>
 Key: metadata = 